Example notebook

In [1]:
import subprocess
import re

r_cell_re = re.compile(r'^```\{r[^\}]+\}')

#def number_of_rmarkdown_cells(rmd_file: str) -> int:
rmd_file = '../demo/notebooks/example_rmarkdown.Rmd'

def number_of_r_cells_in_rmd(rmd_file: str) -> int:
	number_of_r_cells = 0
	with open(rmd_file) as rmd:
		for rmd_line in rmd:
			if r_cell_re.match(rmd_line):
				number_of_r_cells += 1
	return number_of_r_cells

print(number_of_r_cells)

3
